# Building a Decision Tree

The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year.

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
income = pd.read_csv("income.csv", index_col = False)

In [3]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Converting the categorical variables to numeric variables

categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for cat in categorical_cols:
    income[cat] = pd.Categorical(income[cat]).codes

### Calculating Entropy

In [5]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

### Calculating Information Gain

In [6]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

### Finding the best Column to Split On

In [7]:
def find_best_column(data, target_name, columns):
    info_gains = []
    for col in columns:
        info = calc_information_gain(data, col, "high_income")
        info_gains.append(info)
    highest_gain_index = info_gains.index(max(info_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

In [8]:
cols = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

In [9]:
income_split = find_best_column(income, "high_income", cols)
income_split

'marital_status'

### Implementing ID3 algorithm

In [17]:
label_1s = []
label_0s = []

def id3(data, target, columns):
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        if 0 in unique_targets:
            label_0s.append(0)
        elif 1 in unique_targets:
            label_1s.append(1)
        return
    
    best_column = find_best_column(data, target, columns)
    left = data[data[best_column] <= data[best_column].median()]
    right = data[data[best_column] > data[best_column].median()]
    
    for split in [left, right]:
        id3(split, target, columns)

In [20]:
# Testing ID3

data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])

data.columns = ["high_income", "age", "marital_status"]

id3(data, "high_income", ["age", "marital_status"])

### Storing the Tree

In [23]:
tree = {}

nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:        
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
        
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left = data[data[best_column] <= column_median]
    right = data[data[best_column] > column_median]
    split_dict = [["left", left], ["right", right]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

In [24]:
id3(data, "high_income", ["age", "marital_status"], tree)

In [25]:
tree

{'column': 'age',
 'left': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 4},
   'median': 22.5,
   'number': 3,
   'right': {'label': 1, 'number': 5}},
  'median': 25.0,
  'number': 2,
  'right': {'label': 1, 'number': 6}},
 'median': 37.5,
 'number': 1,
 'right': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 9},
   'median': 47.5,
   'number': 8,
   'right': {'label': 1, 'number': 10}},
  'median': 55.0,
  'number': 7,
  'right': {'label': 0, 'number': 11}}}

### Fixing the appearance of the tree

In [26]:
def print_with_depth(string, depth):
    prefix = "    " * depth
    print("{0}{1}".format(prefix, string))

In [27]:
def print_node(tree, depth):
    if "label" in tree:
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        return
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    branches = [tree["left"], tree["right"]]
        
    for b in branches:
        print_node(b,depth+1)

In [28]:
print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


### Making Predictions

In [33]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    
    if row[column]< tree["median"]:
        return predict(tree["left"],row)
    
    if row[column] > tree["median"]:
        return predict(tree["right"],row)

In [34]:
print(predict(tree, data.iloc[0]))

0


### Making multiple predictions

In [35]:
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])

new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)
predictions

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64